In [1]:
import cdsapi
import xarray as xr
import numpy as np
import pandas as pd

# CDS credentials
cdsapi_url = 'https://cds.climate.copernicus.eu/api/v2'
cdsapi_key = '90171b4a-db45-4f29-a1e7-53800377ea00:59c4ecac-f5af-47de-80fc-d9d6cefae125'
c = cdsapi.Client(url=cdsapi_url, key=cdsapi_key)

# Danish offshore point
lat, lon = 56.5, 7.0
delta = 0.05
area = [lat + delta, lon - delta, lat - delta, lon + delta]  # [N, W, S, E]

netcdf_path = 'offshore_dk_hourly_2015_2024.nc'

print("📥 Downloading ERA5 wind data (hourly)...")

c.retrieve(
    'reanalysis-era5-single-levels',
    {
        'product_type': 'reanalysis',
        'variable': ['10m_u_component_of_wind', '10m_v_component_of_wind'],
        'year': [str(y) for y in range(2015, 2025)],
        'month': [f'{m:02d}' for m in range(1, 13)],
        'day': [f'{d:02d}' for d in range(1, 32)],
        'time': [f"{h:02d}:00" for h in range(24)],
        'format': 'netcdf',
        'area': area,
    },
    netcdf_path
)

print("📊 Processing hourly data + interpolating to 10-minute...")

ds = xr.open_dataset(netcdf_path)
ds = ds.mean(dim=['latitude', 'longitude'])
ds['wind_speed'] = np.sqrt(ds['u10']**2 + ds['v10']**2)

df = ds[['u10', 'v10', 'wind_speed']].to_dataframe().reset_index()
df.columns = ['timestamp', 'u10', 'v10', 'wind_speed']

# Set index and interpolate to 10-minute resolution
df = df.set_index('timestamp').resample('10T').interpolate(method='linear').reset_index()

# Save as CSV
csv_path = 'offshore_dk_wind_10min_interpolated.csv'
df.to_csv(csv_path, index=False)

print(f"✅ Done! CSV saved: {csv_path}")


📥 Downloading ERA5 wind data (hourly)...


2025-04-17 10:29:10,505 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels


Exception: Not Found

In [3]:
import cdsapi

c = cdsapi.Client(
    url='https://cds.climate.copernicus.eu/api/v2',
    key='90171b4a-db45-4f29-a1e7-53800377ea00:59c4ecac-f5af-47de-80fc-d9d6cefae125'
)

c.retrieve(
    'reanalysis-era5-land',
    {
        'variable': ['10m_u_component_of_wind', '10m_v_component_of_wind'],
        'year': ['2020'],
        'month': ['01'],
        'day': ['01'],
        'time': ['00:00'],
        'format': 'netcdf',
        'area': [56.55, 6.95, 56.45, 7.05],  # DK offshore box
    },
    'test_land.nc'
)

2025-04-17 10:37:32,335 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-land


Exception: Not Found

In [12]:
import xarray as xr
import numpy as np
import pandas as pd

# === CONFIGURATION ===
# Replace with your actual NetCDF file
input_nc_file = '2024.nc'
output_csv_file = 'wind_speed_100m_2024.csv'

# === LOAD DATA ===
ds = xr.open_dataset(input_nc_file)

# Collapse spatial dimensions (if needed — should be a single point/grid box)
if 'latitude' in ds.dims and 'longitude' in ds.dims:
    ds = ds.mean(dim=['latitude', 'longitude'])

# === COMPUTE WIND SPEED ===
ds['wind_speed_100m'] = np.sqrt(ds['u100']**2 + ds['v100']**2)

# === EXPORT TO CSV ===
# Extract relevant variables
df = ds[['u100', 'v100', 'wind_speed_100m']].to_dataframe().reset_index()

# Rename timestamp for clarity
df.rename(columns={'time': 'timestamp'}, inplace=True)

# Save to CSV
df.to_csv(output_csv_file, index=False)

print(f"✅ Export complete: {output_csv_file}")

✅ Export complete: wind_speed_100m_2024.csv


In [5]:
import cdsapi

c = cdsapi.Client(
    url='https://cds.climate.copernicus.eu/api/v2',
    key='90171b4a-db45-4f29-a1e7-53800377ea00:59c4ecac-f5af-47de-80fc-d9d6cefae125'
)

c.retrieve(
    'reanalysis-era5-land',
    {
        'variable': ['10m_u_component_of_wind', '10m_v_component_of_wind'],
        'year': ['2020'],
        'month': ['01'],
        'day': ['01'],
        'time': ['00:00'],
        'format': 'netcdf',
        'area': [56.55, 6.95, 56.45, 7.05],  # DK offshore box
    },
    'test_land.nc'
)

2025-04-17 11:04:46,511 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-land


Exception: Not Found

In [7]:
import cdsapi

dataset = "reanalysis-era5-single-levels"
request = {
    "product_type": ["reanalysis"],
    "variable": [
        "100m_u_component_of_wind",
        "100m_v_component_of_wind"
    ],
    "year": ["2016"],
    "month": [
        "01", "02", "03",
        "04", "05", "06",
        "07", "08", "09",
        "10", "11", "12"
    ],
    "day": [
        "01", "02", "03",
        "04", "05", "06",
        "07", "08", "09",
        "10", "11", "12",
        "13", "14", "15",
        "16", "17", "18",
        "19", "20", "21",
        "22", "23", "24",
        "25", "26", "27",
        "28", "29", "30",
        "31"
    ],
    "time": [
        "00:00", "01:00", "02:00",
        "03:00", "04:00", "05:00",
        "06:00", "07:00", "08:00",
        "09:00", "10:00", "11:00",
        "12:00", "13:00", "14:00",
        "15:00", "16:00", "17:00",
        "18:00", "19:00", "20:00",
        "21:00", "22:00", "23:00"
    ],
    "data_format": "netcdf",
    "download_format": "unarchived",
    "area": [55.6, 7.48, 55.5, 7.58]
}

client = cdsapi.Client()
client.retrieve(dataset, request).download()


2025-04-17 11:10:22,217 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels


Exception: Not Found

In [8]:
import cdsapi

client = cdsapi.Client()

client.retrieve(
    'reanalysis-era5-single-levels',
    {
        'product_type': 'reanalysis',
        'variable': ['100m_u_component_of_wind', '100m_v_component_of_wind'],
        'year': '2020',
        'month': '01',
        'day': '01',
        'time': ['00:00', '01:00', '02:00'],  # keep it tiny for test
        'format': 'netcdf',
        'area': [55.6, 7.48, 55.5, 7.58],  # Horns Rev offshore
    }
).download('test_100m.nc')

2025-04-17 11:11:34,400 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels


Exception: Not Found

In [13]:
import xarray as xr
import numpy as np
import pandas as pd

# === CONFIGURATION ===
input_files = [
    '2015.nc', '2016.nc', '2017.nc', '2018.nc', '2019.nc',
    '2020.nc', '2021.nc', '2022.nc', '2023.nc', '2024.nc'
]
output_csv_file = 'wind_speed_100m_10min_2015_2024.csv'

# === PROCESS FILES ===
all_dfs = []

for input_file in input_files:
    print(f"📂 Processing {input_file}...")

    ds = xr.open_dataset(input_file)

    # Collapse to a point (if it's a small box)
    if 'latitude' in ds.dims and 'longitude' in ds.dims:
        ds = ds.mean(dim=['latitude', 'longitude'])

    # Compute wind speed
    ds['wind_speed_100m'] = np.sqrt(ds['u100']**2 + ds['v100']**2)

    # Convert to DataFrame
    df = ds[['wind_speed_100m']].to_dataframe().reset_index()
    df.rename(columns={'time': 'timestamp'}, inplace=True)

    # Interpolate to 10-minute resolution
    df.set_index('timestamp', inplace=True)
    df = df.resample('10T').interpolate(method='linear')  # Linear interpolation
    df = df.reset_index()

    # Keep only relevant columns
    all_dfs.append(df[['timestamp', 'wind_speed_100m']])

# === COMBINE AND EXPORT ===
final_df = pd.concat(all_dfs).sort_values(by='timestamp').reset_index(drop=True)
final_df.to_csv(output_csv_file, index=False)

print(f"✅ All done! 10-min CSV saved to: {output_csv_file}")

📂 Processing 2015.nc...


KeyError: "None of ['timestamp'] are in the columns"

In [14]:
import xarray as xr
import numpy as np
import pandas as pd

# === CONFIGURATION ===
input_files = [
    '2015.nc', '2016.nc', '2017.nc', '2018.nc', '2019.nc',
    '2020.nc', '2021.nc', '2022.nc', '2023.nc', '2024.nc'
]
output_csv_file = 'wind_speed_100m_10min_2015_2024.csv'

# === PROCESS FILES ===
all_dfs = []

for input_file in input_files:
    print(f"📂 Processing {input_file}...")

    ds = xr.open_dataset(input_file)

    # Collapse to point if it's a small box
    if 'latitude' in ds.dims and 'longitude' in ds.dims:
        ds = ds.mean(dim=['latitude', 'longitude'])

    # Compute wind speed
    ds['wind_speed_100m'] = np.sqrt(ds['u100']**2 + ds['v100']**2)

    # Determine time column name
    time_col = 'valid_time' if 'valid_time' in ds.coords else 'time'

    # Convert to DataFrame
    df = ds[['wind_speed_100m']].to_dataframe().reset_index()

    # Rename time column to 'timestamp'
    df.rename(columns={time_col: 'timestamp'}, inplace=True)

    # Interpolate to 10-minute resolution
    df.set_index('timestamp', inplace=True)
    df = df.resample('10T').interpolate(method='linear')
    df = df.reset_index()

    # Keep only timestamp and wind_speed_100m
    all_dfs.append(df[['timestamp', 'wind_speed_100m']])

# === COMBINE AND EXPORT ===
final_df = pd.concat(all_dfs).sort_values(by='timestamp').reset_index(drop=True)
final_df.to_csv(output_csv_file, index=False)

print(f"✅ All done! 10-min CSV saved to: {output_csv_file}")


📂 Processing 2015.nc...
📂 Processing 2016.nc...
📂 Processing 2017.nc...
📂 Processing 2018.nc...


C:\Users\gepap\AppData\Local\Temp\ipykernel_3312\268867624.py:38: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df = df.resample('10T').interpolate(method='linear')
C:\Users\gepap\AppData\Local\Temp\ipykernel_3312\268867624.py:38: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df = df.resample('10T').interpolate(method='linear')
C:\Users\gepap\AppData\Local\Temp\ipykernel_3312\268867624.py:38: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df = df.resample('10T').interpolate(method='linear')
C:\Users\gepap\AppData\Local\Temp\ipykernel_3312\268867624.py:38: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df = df.resample('10T').inter

📂 Processing 2019.nc...
📂 Processing 2020.nc...
📂 Processing 2021.nc...
📂 Processing 2022.nc...
📂 Processing 2023.nc...


C:\Users\gepap\AppData\Local\Temp\ipykernel_3312\268867624.py:38: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df = df.resample('10T').interpolate(method='linear')
C:\Users\gepap\AppData\Local\Temp\ipykernel_3312\268867624.py:38: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df = df.resample('10T').interpolate(method='linear')
C:\Users\gepap\AppData\Local\Temp\ipykernel_3312\268867624.py:38: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df = df.resample('10T').interpolate(method='linear')
C:\Users\gepap\AppData\Local\Temp\ipykernel_3312\268867624.py:38: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df = df.resample('10T').inter

📂 Processing 2024.nc...
✅ All done! 10-min CSV saved to: wind_speed_100m_10min_2015_2024.csv
